In [29]:
!pip install -q efficientnet
from keras.models import load_model
import os, re, time, tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics, model_selection
from kaggle_datasets import KaggleDatasets
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras 
from tensorflow.keras import backend as K
from efficientnet import tfkeras as efnet

In [69]:
EFF_NET = 0
IMG_SIZE =384

LOSS_PARAMS  = dict(label_smoothing=0.09)
MODEL_WEIGHT_PATH='../model_B0/effcientB0.h5'

# Model Loading

In [13]:
def build_model(dim=128, ef=0):
    inp = keras.layers.Input(shape=(dim,dim,3))
    base = getattr(efnet, 'EfficientNetB%d' % ef)(input_shape=(dim, dim, 3), weights='imagenet', include_top=False)
    x = base(inp)
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.BatchNormalization()(x)
    #x = keras.layers.Dense(1024)(x)
    #x = keras.layers.Dropout(0.2)(x)
    x = keras.layers.Dense(512,activation='relu')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dense(256,activation='relu')(x)
#     x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(256,activation='relu')(x)
    x = keras.layers.Dropout(0.3)(x)
#     x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dense(128,activation='relu')(x)
    x = keras.layers.Dropout(0.3)(x)

    x = keras.layers.Dense(1)(x)
    x = keras.layers.Activation('sigmoid', dtype='float32')(x)
    model = keras.Model(inputs=inp,outputs=x)
    opt = keras.optimizers.Adam(learning_rate=1e-3)
#     opt = keras.optimizers.SGD(learning_rate=1e-3)

    loss = keras.losses.BinaryCrossentropy(**LOSS_PARAMS)

    model.compile(optimizer=opt, loss=loss, metrics=['AUC'])
    return model

In [70]:
model = build_model(dim=IMG_SIZE, ef=EFF_NET)
model.load_weights(MODEL_WEIGHT_PATH)

# Dataset Preapring

In [67]:
def read_labeled_tfrecord(example):
    tfrec_format = {
        'image'                        : tf.io.FixedLenFeature([], tf.string),
        'image_name'                   : tf.io.FixedLenFeature([], tf.string),
        'target'                       : tf.io.FixedLenFeature([], tf.int64)
    }           
    example = tf.io.parse_single_example(example, tfrec_format)
    return example['image'], example['target']

def read_unlabeled_tfrecord(example, return_image_name=True):
    tfrec_format = {
        'image'                        : tf.io.FixedLenFeature([], tf.string),
        'image_name'                   : tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example, tfrec_format)
    return example['image'], example['image_name'] if return_image_name else 0

def prepare_image(img):    
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32) / 255.0           
    return img

In [96]:
GCS_PATH = KaggleDatasets().get_gcs_path('isic2019-%ix%i' % (IMG_SIZE, IMG_SIZE))
test_files = tf.io.gfile.glob(os.path.join(GCS_PATH, "train%.2i*.tfrec" % i) for i in range(1, 30, 2))
test_dataset = tf.data.TFRecordDataset(test_files)
test_dataset = test_dataset.cache()
test_dataset = test_dataset.map(read_labeled_tfrecord)
test_dataset = test_dataset.map(lambda img, o: (prepare_image(img), o))
test_dataset = test_dataset.map(lambda i, o: (tf.image.resize(i, [IMG_SIZE, IMG_SIZE]), o))
test_dataset = test_dataset.batch(15, drop_remainder=True)

In [97]:
test_dataset

<BatchDataset shapes: ((15, 384, 384, 3), (15,)), types: (tf.float32, tf.int64)>

# Inference Part

In [98]:
TEST_STEPS = 15

test_pred = model.predict(test_dataset.map(lambda img, l: img), steps=TEST_STEPS, verbose=1)

1/1 [==============================] - 0s 2ms/step


In [94]:
test_pred

array([[0.19490618],
       [0.0682652 ],
       [0.3129686 ],
       [0.2540205 ],
       [0.07658222]], dtype=float32)